In [1]:
#reading data
import gzip #to read gzip files

#manipulating data 
import pandas as pd
import numpy as np
import datetime

#data path
PATH = '../'

## Data extraction 

We start by extracting the data for beers, breweries and users from the two rating websites BeerAdvocate and RateBeer as well as the matched data. <br>
We view the dataframe first few rows to get a firt glance at what the data contains before pre-processing it:

### BeerAdvocate data 

In [5]:
BA_beers = pd.read_csv(PATH+'BeerAdvocate/beers.csv', index_col='beer_id')
BA_breweries = pd.read_csv(PATH+'BeerAdvocate/breweries.csv', index_col='id')
BA_users = pd.read_csv(PATH+'BeerAdvocate/users.csv', index_col= 'user_id')

As for the BeerAdvocate text reviews, the text files were in a quite special format, we extracted them in the notebook 'TransformTextfileToCsv.ipynb' ([here](TransformTextfileToCsv.ipynb)) and saved them into csv files which we load below to take a look at: 

In [4]:
#open the csv file
df_BA_reviews = pd.read_csv('../DataframeStorage/df_BA_reviews_unprocessed.csv')

### RateBeer data 

In [6]:
RB_beers = pd.read_csv(PATH+'RateBeer/beers.csv', index_col='beer_id')
RB_breweries = pd.read_csv(PATH+'RateBeer/breweries.csv', index_col= 'id')
RB_users = pd.read_csv(PATH+'RateBeer/users.csv', index_col= 'user_id')

For the RateBeer text reviews, the text files were in a quite special format, we extracted them in the notebook 'TransformTextfileToCsv.ipynb' ([here](TransformTextfileToCsv.ipynb)) and saved them into csv files:

In [7]:
#open the csv file
df_RB_reviews = pd.read_csv('../DataframeStorage/df_RB_reviews_unprocessed.csv')

### Matched data 

In [8]:
matched_beers = pd.read_csv(PATH+'matched_beer_data/beers.csv', header=1)
matched_breweries = pd.read_csv(PATH+'matched_beer_data/breweries.csv', header=1)
matched_users = pd.read_csv(PATH+'matched_beer_data/users.csv', header=1)

In [9]:
matched_ratings = pd.read_csv(PATH+'matched_beer_data/ratings.csv', encoding = "ISO-8859-1", header=1)

## Pre-processing

In this part we were interested in modifying the reviews dataframe in such a way that it contains columns containing the user and brewery locations. But this required different preprocessing steps on users and breweries first in order to match them afterwards in the review data.

### Users

For both websites :
- We convert the *joined* column from seconds into a datetime object 
- We add a column called *merged_location* where all the states of the United States are just defined as United States (this will be useful for analyses where we only care about comparing countries)

In [ ]:
BA_users['joined'] = pd.to_datetime(BA_users['joined'],unit = 's')

# Add a column where all Users from the united states get 'United States' as location
BA_users['location'] = BA_users['location'].fillna('Unknown')
BA_users['merged_location'] = BA_users['location'].copy()
BA_users.loc[(BA_users['merged_location'].str.startswith('United States')), 'merged_location'] = 'United States'

In [ ]:
RB_users['joined'] = pd.to_datetime(RB_users['joined'] ,unit = 's')

# Add a column where all Users from the united states get 'United States' as location
RB_users['location'] = RB_users['location'].fillna('Unknown')
RB_users['merged_location'] = RB_users['location'].copy()
RB_users.loc[(RB_users['merged_location'].str.startswith('United States')), 'merged_location'] = 'United States'

For the matched data we just convert the *joined* to a datatime object as we will use primarly the two other dataframes when it comes to analyzing locations:

In [ ]:
matched_users['joined'] = pd.to_datetime(matched_users['joined'],unit = 's')
matched_users['joined.1'] = pd.to_datetime(matched_users['joined.1'],unit = 's')

### Breweries 

We proceed similarly with the two website breweries:

In [ ]:
# Add a column where all Users from the united states get 'United States' as location
RB_breweries['merged_location'] = RB_breweries['location'].copy()
RB_breweries.loc[(RB_breweries['merged_location'].str.startswith('United States')), 'merged_location'] = 'United States'

In [ ]:
BA_breweries['merged_location'] = BA_breweries['location'].copy()
BA_breweries.loc[(BA_breweries['merged_location'].str.startswith('United States')), 'merged_location'] = 'United States'

### Reviews 

To facilitate consequent analyses, we wanted to have both the user location and the brewery location as columns in the review data, we proceed to do the following:
To allow the merge of the brewery location with the brewery name on df_reviews, we first need to align the column names. <br>
To do so we rename the column "name" of breweries to "brewery_name" (the name of the column in df_reviews). <br>

We use the function df.merge to add the element "location" to the corresponding brewery in df_reviews. <br>
By default the new columns name would be the same as in df_breweries ('location'), we rename it by "brewery_location" to prevent confusion. <br>

After the merging we rename the column of the brewery dataframe again, to have the same name that we had initially.

In [ ]:
#Rename the column name to brewery_name to allow merging with the 
BA_breweries.rename(columns = {'name':'brewery_name'}, inplace = True)
RB_breweries.rename(columns = {'name':'brewery_name'}, inplace = True)


#Add the location to the beer dataframe
df_BA_reviews = (df_BA_reviews.merge(BA_breweries[['location', 'brewery_name']], on=['brewery_name'], how='left')).rename(columns = {'location':'brewery_location'})
df_RB_reviews = (df_RB_reviews.merge(RB_breweries[['location', 'brewery_name']], on=['brewery_name'], how='left')).rename(columns = {'location':'brewery_location'})


#Name the columns back: 
BA_breweries.rename(columns = {'brewery_name':'name'}, inplace = True)
RB_breweries.rename(columns = {'brewery_name':'name'}, inplace = True)

As the different states of the US are described as different locations, but sometimes we want to group the beers by nations and not by states. <br>
To make this possible we add a new column. <br>
This column is called "brewery_merged_location" and is exactly the same as the column "brewery_location" except for the fact, that the different states of the US all take the value "United States".

In [ ]:
# Add a column where all breweries from the united states get 'United States' as location
df_RB_reviews['brewery_merged_location'] = df_RB_reviews['brewery_location'].copy()
df_RB_reviews.loc[(df_RB_reviews['brewery_merged_location'].str.startswith('United States')), 'brewery_merged_location'] = 'United States'


df_BA_reviews['brewery_merged_location'] = df_BA_reviews['brewery_location'].copy()
df_BA_reviews.loc[(df_BA_reviews['brewery_merged_location'].str.startswith('United States')), 'brewery_merged_location'] = 'United States'

The same approach as for the brewery locations is used to add the country of origin of the users. <br>
To group them by nations we add here a column that sums up the states of the US under the value "United States".

In [ ]:
#Add the country of origin of the reviewer to the review
df_RB_reviews = (df_RB_reviews.merge(RB_users[['location', 'user_name']], on=['user_name'], how='left')).rename(columns={'location':'user_location'})
df_BA_reviews = (df_BA_reviews.merge(BA_users[['location', 'user_name']], on=['user_name'], how='left')).rename(columns={'location':'user_location'})

# Add a column where all users from the united states get 'United States' as location
df_RB_reviews['user_location'] = df_RB_reviews['user_location'].fillna('Unknown')
df_RB_reviews['user_merged_location'] = df_RB_reviews['user_location'].copy()
df_RB_reviews.loc[(df_RB_reviews['user_merged_location'].str.startswith('United States')), 'user_merged_location'] = 'United States'

#For BA we have to replace the NaN values
df_BA_reviews['user_location'] = df_BA_reviews['user_location'].fillna('Unknown')
df_BA_reviews['user_merged_location'] = df_BA_reviews['user_location'].copy()
df_BA_reviews.loc[(df_BA_reviews['user_merged_location'].str.startswith('United States')), 'user_merged_location'] = 'United States'

In [ ]:
#Transform the dates of the reviews from string to datetime
df_BA_reviews['date'] = pd.to_datetime(df_BA_reviews['date'], format="%Y-%m-%d %H:%M:%S")
df_RB_reviews['date'] = pd.to_datetime(df_RB_reviews['date'], format="%Y-%m-%d %H:%M:%S")

### Changing format of some of the locations :

We've noticed that a big part of the US states reviews in the BeerAdvocate data have links to the brewery as a brewery_location, we count how many datapoints are in each of these long strings (that's how we identify they are links):

In [ ]:
BA_brewery_locations = pd.DataFrame(df_BA_reviews['brewery_location'].value_counts())
BA_brewery_locations.loc[BA_brewery_locations.index.str.len()>40]

,brewery_location
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://shipyard.com"" target=""_blank"">shipyard.com",5884
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://mendobrew.com"" target=""_blank"">mendobrew.com",4060
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://thirstydog.com"" target=""_blank"">thirstydog.com",3213
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://rockbottom.com"" target=""_blank"">rockbottom.com",2685
"Utah</a><br><a href=""http://utahbeers.com"" target=""_blank"">utahbeers.com",2503
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://ironhillbrewery.com"" target=""_blank"">ironhillbrewery.com",2238
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://seadogbrewing.com"" target=""_blank"">seadogbrewing.com",1182
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://sebagobrewing.com"" target=""_blank"">sebagobrewing.com",1101
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://gcfb.net"" target=""_blank"">gcfb.net",386
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://hopsonline.com"" target=""_blank"">hopsonline.com",167


So we convert these to their respective States in a manual way (by going to the referred links):

In [ ]:
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://thirstydog.com" target="_blank">thirstydog.com', 'brewery_location'] = 'United States, Ohio'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://mendobrew.com" target="_blank">mendobrew.com', 'brewery_location'] = 'United States, California'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://shipyard.com" target="_blank">shipyard.com', 'brewery_location'] = 'United States, Maine'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://sebagobrewing.com" target="_blank">sebagobrewing.com', 'brewery_location'] = 'United States, Maine'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://herefordandhops.com" target="_blank">herefordandhops.com', 'brewery_location'] = 'United States, Michigan'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://steelheadbrewingco.com" target="_blank">steelheadbrewingco.com', 'brewery_location'] = 'United States, Oregon'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://bigbuck.com" target="_blank">bigbuck.com', 'brewery_location'] = 'United States, Michigan'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://bluecorncafe.com" target="_blank">bluecorncafe.com', 'brewery_location'] = 'United States, New Mexico'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='United States</a> | <a href="http://maps.google.com/maps?oi=map&q=%2C+US" target="_blank">map</a><br><a href="http://capcitybrew.com" target="_blank">capcitybrew.com', 'brewery_location'] = 'United States, Washington'


df_BA_reviews.loc[df_BA_reviews['brewery_location']=='Utah</a><br><a href="http://utahbeers.com" target="_blank">utahbeers.com', 'brewery_location'] = 'United States, Utah'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='Illinois</a>, 60614-4939, <a href="/place/directory/9/US/">United States', 'brewery_location'] = 'United States, Illinois'
df_BA_reviews.loc[df_BA_reviews['brewery_location']=='New York</a>, 13057, <a href="/place/directory/9/US/">United States', 'brewery_location'] = 'United States, New York'

We re-run the following command because some of the links that were converted above didn't start wiht 'United States', but with the State name:

In [ ]:
df_BA_reviews.loc[(df_BA_reviews['brewery_location'].str.startswith('United States')), 'brewery_merged_location'] = 'United States'

However there are some links (which have not been converted into their respective locations above) which refer to breweries that have many locations across many states in the US so we decided to change their location to 'United States, Different States'. Those are the following links:

In [ ]:
BA_brewery_locations = pd.DataFrame(df_BA_reviews['brewery_location'].value_counts())
BA_brewery_locations.loc[BA_brewery_locations.index.str.len()>40]

,brewery_location
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://rockbottom.com"" target=""_blank"">rockbottom.com",2685
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://ironhillbrewery.com"" target=""_blank"">ironhillbrewery.com",2238
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://seadogbrewing.com"" target=""_blank"">seadogbrewing.com",1182
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://gcfb.net"" target=""_blank"">gcfb.net",386
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://hopsonline.com"" target=""_blank"">hopsonline.com",167
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map",104
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://tiedhouse.com"" target=""_blank"">tiedhouse.com",80
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://theram.com"" target=""_blank"">theram.com",68
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://buckheadbrewery.com"" target=""_blank"">buckheadbrewery.com",24
"United States</a> | <a href=""http://maps.google.com/maps?oi=map&q=%2C+US"" target=""_blank"">map</a><br><a href=""http://eotrading.com"" target=""_blank"">eotrading.com",13


In [ ]:
df_BA_reviews.loc[df_BA_reviews['brewery_location'].str.startswith('United States</a>'), 'brewery_location']='United States, Different States'

### Normalizing all ratings in a scale of 5

In [ ]:
df_BA_scaled = df_BA_reviews

df_RB_reviews['aroma'] = df_RB_reviews['aroma'].transform(lambda x: x/2)
df_RB_reviews['taste'] = df_RB_reviews['taste'].transform(lambda x: x/2)
df_RB_reviews['overall'] = df_RB_reviews['overall'].transform(lambda x: x/4)
# appearance is already with the correct scale


df_RB_scaled = df_RB_reviews

## Saving the now pre-processed data:

To avoid doing these heavy computations (each take a long time) several times, we saved those dataframes as mentionned earlier; <br>

In [ ]:
#Do that only once. Stores the dataframes so that we can access them more easily next time and have to do the data preprocessing only once
df_BA_reviews.to_csv(PATH+'/DataframeStorage/df_BA_reviews.csv', columns=['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 'rating', 'text', 'brewery_location', 'brewery_merged_location', 'user_location', 'user_merged_location'], index=False)

: 

In [ ]:
df_RB_reviews.to_csv(PATH+'/DataframeStorage/df_RB_reviews.csv', columns=['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 'rating', 'text', 'brewery_location', 'brewery_merged_location', 'user_location', 'user_merged_location'], index=False)

In [ ]:
df_reviews_all = pd.concat([df_BA_reviews, df_RB_reviews], axis=0)

df_reviews_all.to_csv(PATH+'/DataframeStorage/df_reviews_all.csv', columns=['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 'rating', 'text', 'brewery_location', 'brewery_merged_location', 'user_location', 'user_merged_location'], index=False)

In [ ]:
df_all_scaled = pd.concat([df_BA_scaled, df_RB_scaled], axis=0)

df_all_scaled.to_csv('DataframeStorage/df_all_scaled.csv', columns=['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall', 'rating', 'text', 'brewery_location', 'brewery_merged_location', 'user_location', 'user_merged_location'], index=False)